In [ ]:
# === Environment Setup ===
import os, sys, math, time, random, json, textwrap, warnings
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from scipy.optimize import fsolve
from IPython.display import display, Markdown, Image

# --- Configuration ---
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({'font.size': 12, 'figure.figsize': (10, 6), 'figure.dpi': 130,
                     'axes.titlesize': 'x-large', 'axes.labelsize': 'large',
                     'xtick.labelsize': 'medium', 'ytick.labelsize': 'medium'})
np.set_printoptions(suppress=True, linewidth=120, precision=4)

# --- Utility Functions ---
def note(msg, **kwargs):
    display(Markdown(f"<div class='alert alert-block alert-info'>📝 **Note:** {msg}</div>"))
def sec(title):
    print(f"\n{100*'='}\n| {title.upper()} |\n{100*'='}")

note("Environment initialized for advanced New Keynesian model analysis.")

# Part 4: Macroeconomic Models
## Chapter 4.05: New Keynesian (NK) Models: The Modern Framework for Monetary Policy

### Table of Contents
1.  [The New Keynesian Synthesis](#1.-The-New-Keynesian-Synthesis)
2.  [Microfoundations of the Three-Equation Model](#2.-Microfoundations-of-the-Three-Equation-Model)
    *   [2.1 The Dynamic IS Curve](#2.1-The-Dynamic-IS-Curve)
    *   [2.2 The New Keynesian Phillips Curve (NKPC)](#2.2-The-New-Keynesian-Phillips-Curve-(NKPC))
    *   [2.3 The Monetary Policy Rule](#2.3-The-Monetary-Policy-Rule)
3.  [Solving and Analyzing the Canonical Model](#3.-Solving-and-Analyzing-the-Canonical-Model)
    *   [3.1 The Model in Matrix Form](#3.1-The-Model-in-Matrix-Form)
    *   [3.2 Impulse Response to a Monetary Policy Shock](#3.2-Impulse-Response-to-a-Monetary-Policy-Shock)
4.  [Optimal Monetary Policy](#4.-Optimal-Monetary-Policy)
5.  [The Zero Lower Bound (ZLB)](#5.-The-Zero-Lower-Bound-(ZLB))
6.  [Chapter Summary](#6.-Chapter-Summary)
7.  [Exercises](#7.-Exercises)

### 1. The New Keynesian Synthesis

While the RBC model provided a powerful, microfounded framework, its core assumption that business cycles are efficient responses to technology shocks struggled to explain several key empirical facts, particularly the observation that purely nominal variables (like the money supply) seemed to have significant real effects. The **New Keynesian (NK) model** emerged in the 1990s as a response. It represents the modern synthesis in macroeconomics, blending the methodological rigor of the RBC school with the core insights of the Keynesian tradition—namely, that market imperfections can lead to inefficient fluctuations that can be addressed by stabilization policy.

The model retains the DSGE framework—complete with rational expectations and intertemporal optimization—but introduces a crucial friction: **nominal rigidities**. The most common form is **sticky prices**, where firms are assumed to be unable to change their prices instantly in response to shocks. This single, micro-founded friction is revolutionary. It breaks the neutrality of money and creates a coherent framework where monetary policy actions have real effects on output and employment in the short run.

### 2. Microfoundations of the Three-Equation Model
The canonical NK model can be boiled down to three core equations, each derived from the optimization problems of households and firms.

#### 2.1 The Dynamic IS Curve (Aggregate Demand)
This equation is the log-linearized Euler equation from the representative household's consumption-smoothing problem.
$$ x_t = E_t[x_{t+1}] - \frac{1}{\sigma}(\hat{i}_t - E_t[\hat{\pi}_{t+1}] - r_t^n) $$    
Here, $x_t$ is the **output gap**, $\hat{i}_t$ and $\hat{\pi}_t$ are deviations of the nominal rate and inflation from their targets, $1/\sigma$ is the intertemporal elasticity of substitution, and $r_t^n$ is the **natural rate of interest**. It states that the output gap today is negatively related to the gap between the real interest rate and its natural level.

#### 2.2 The New Keynesian Phillips Curve (Aggregate Supply)
This equation is derived from monopolistically competitive firms that face **Calvo (1983) pricing frictions**, meaning only a fraction $(1-\theta)$ of firms can reset their price each period. A firm that can reset its price chooses $P_t^*$ to maximize the present discounted value of its expected future profits. Aggregating across all firms and log-linearizing yields the **New Keynesian Phillips Curve (NKPC)**:
$$ \hat{\pi}_t = \beta E_t[\hat{\pi}_{t+1}] + \kappa x_t + u_t $$
Inflation $\hat{\pi}_t$ depends on expected future inflation and the output gap $x_t$ (which is proportional to real marginal cost). The slope, $\kappa$, is a composite parameter that depends on the degree of price stickiness $\theta$, the discount factor $\beta$, and other preference parameters.

#### 2.3 The Monetary Policy Rule
We close the model with a **Taylor (1993) rule**, which posits that the central bank systematically responds to deviations of inflation and the output gap from their targets (assumed to be zero).
$$ \hat{i}_t = \phi_\pi \hat{\pi}_t + \phi_y x_t + v_t $$
For the model to have a unique, stable equilibrium, the central bank must adhere to the **Taylor Principle**, which requires $\phi_\pi > 1$. The intuition is crucial: if inflation rises by 1 percentage point, the central bank must raise the nominal interest rate by *more* than 1 percentage point. This ensures that the *real* interest rate ($r_t = i_t - E_t[\pi_{t+1}]$) also rises, which is necessary to cool down the economy and bring inflation back to target. If the central bank were to raise the nominal rate by less than the increase in inflation ($"phi"_\pi < 1$), the real rate would actually fall, stimulating the economy and leading to an explosive, self-fulfilling spiral of ever-increasing inflation.

### 3. Solving and Analyzing the Canonical Model

#### 3.1 The Model in Matrix Form
To solve the model, we write the system of three linear rational expectations equations in the standard matrix form $\mathbf{A} E_t[\mathbf{y}_{t+1}] = \mathbf{B} \mathbf{y}_t + \mathbf{C} \mathbf{v}_t$, where $\mathbf{y}_t = [x_t, \pi_t, i_t]'$ is the vector of endogenous variables and $v_t$ is the monetary policy shock. This system can be solved using standard methods like Uhlig's (1999) algorithm.

In [ ]:
sec("Solving and Simulating the 3-Equation NK Model")

class NKModel:
    """
    Solves and simulates the canonical 3-equation New Keynesian model.
    Parameters:
      - beta: Subjective discount factor
      - sigma: Coefficient of relative risk aversion (CRRA)
      - phi: Inverse of the Frisch elasticity of labor supply
      - theta: Calvo parameter (degree of price stickiness)
      - phi_pi: Taylor rule response to inflation
      - phi_y: Taylor rule response to the output gap
      - rho_v: Persistence of the monetary policy shock
      - shock_size: Size of the monetary policy shock
    """
    def __init__(self, beta=0.99, sigma=1.0, phi=1.0, theta=0.75, phi_pi=1.5, phi_y=0.1, rho_v=0.5, shock_size=0.0025):
        self.beta, self.sigma, self.phi, self.theta = beta, sigma, phi, theta
        self.phi_pi, self.phi_y = phi_pi, phi_y
        self.rho_v, self.shock_size = rho_v, shock_size
        # kappa is the slope of the NK Phillips Curve
        self.kappa = ((1 - self.theta) * (1 - self.beta * self.theta) / self.theta) * (self.sigma + self.phi)
        self.solution = self._solve()

    def _solve(self):
        # This uses the method of undetermined coefficients for this specific simple case.
        denom = self.sigma * (1 - self.rho_v) + self.phi_y + (self.kappa * (self.phi_pi - self.rho_v)) / (1 - self.beta * self.rho_v)
        Px = -1 / denom
        Ppi = (self.kappa / (1 - self.beta * self.rho_v)) * Px
        Pi = self.phi_pi * Ppi + self.phi_y * Px + 1
        return {'x': Px, 'pi': Ppi, 'i': Pi}

    def generate_irfs(self, T=40):
        v_path = np.zeros(T); v_path[0] = self.shock_size
        for t in range(T - 1): v_path[t+1] = self.rho_v * v_path[t]
        
        x_path = self.solution['x'] * v_path
        pi_path = self.solution['pi'] * v_path
        i_path = self.solution['i'] * v_path
        
        # The ex-ante real rate is i_t - E_t[pi_{t+1}]
        # Since E_t[pi_{t+1}] = P_pi * E_t[v_{t+1}] = P_pi * rho_v * v_t
        pi_exp_path = self.solution['pi'] * self.rho_v * v_path
        r_path = i_path - pi_exp_path
        
        return 100 * pd.DataFrame({'Output Gap': x_path, 'Inflation': pi_path, 
                                     'Nominal Rate': i_path, 'Real Rate': r_path})

nk_model = NKModel()
irfs = nk_model.generate_irfs()
note(f"Model parameters: β={nk_model.beta}, σ={nk_model.sigma}, φ={nk_model.phi}, θ={nk_model.theta}, κ={nk_model.kappa:.3f}")

#### 3.2 Impulse Response to a Monetary Policy Shock
A positive shock $v_t$ represents an unexpected, contractionary monetary policy action (the central bank raises rates more than the Taylor rule prescribes). The economy responds as follows:
- The **nominal and real interest rates** rise.
- The higher real rate reduces aggregate demand, causing a negative **output gap** (a recession).
- The fall in the output gap (lower marginal costs) reduces **inflation**.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 9))
irfs.plot(subplots=True, ax=axes, legend=False, style='-o', markersize=4)
for i, ax in enumerate(axes.flatten()):
    ax.set_title(irfs.columns[i]); ax.axhline(0, color='k', lw=0.7, ls='--')
    ax.set_ylabel('% dev. from s.s.'); ax.set_xlabel('Quarters after shock')
fig.suptitle('Impulse Responses to a Contractionary Monetary Policy Shock', fontsize=16, y=1.02)
fig.tight_layout()

### 4. Optimal Monetary Policy
Instead of assuming the central bank follows a simple rule, we can solve for the **optimal policy**. We assume the central bank minimizes a quadratic loss function that penalizes volatility in inflation and the output gap:
$$ \mathcal{L} = E_0 \sum_{t=0}^\infty \beta^t (\pi_t^2 + \lambda_x x_t^2) $$
Minimizing this loss subject to the constraint of the NKPC yields the **optimal targeting rule**. Under commitment, the FOC for this problem implies a relationship that must hold in all periods:
$$ x_t - x_{t-1} = -\frac{\kappa}{\lambda_x} \pi_t $$
This rule dictates that the central bank should engineer a fall in the output gap whenever inflation is above target. It implies a trade-off: to reduce inflation by one unit, the bank must accept a cost of $\lambda_x/\kappa$ units of lost output. This is the **sacrifice ratio**.

### 5. The Zero Lower Bound (ZLB)
A crucial challenge for modern monetary policy is the **Zero Lower Bound (ZLB)** on nominal interest rates. When a large negative shock hits the economy, the natural rate of interest $r_t^n$ can become deeply negative. The optimal policy might prescribe setting $i_t < 0$, but this is not possible. The policy rule becomes constrained: $i_t = \max(0, \phi_\pi \pi_t + \phi_y x_t)$.

When stuck at the ZLB, the central bank loses its primary tool for stimulating the economy. If the recession leads to deflation ($"pi"_t < 0$), the real interest rate ($i_t - \pi_t \approx -\pi_t$) actually *rises*, further depressing the economy and potentially leading to a dangerous deflationary spiral. To solve this non-linear model, we must use different techniques, such as the perfect-foresight time iteration method shown below.

In [ ]:
sec("Simulating a ZLB Episode")
def simulate_zlb(params, r_n_path):
    T = len(r_n_path)
    x_path, pi_path, i_path = np.zeros(T), np.zeros(T), np.zeros(T)
    # Iterate backwards from the terminal steady state (where x=pi=0)
    for t in range(T - 2, -1, -1):
        x_exp, pi_exp = x_path[t+1], pi_path[t+1]
        # Solve the system of two equations for x_t, pi_t
        def system(vars):
            x, pi = vars
            i = max(0, params['phi_pi']*pi + params['phi_y']*x) # ZLB constraint
            eq1 = x - x_exp + (1/params['sigma'])*(i - pi_exp - r_n_path[t])
            eq2 = pi - params['beta']*pi_exp - params['kappa']*x
            return [eq1, eq2]
        x_path[t], pi_path[t] = fsolve(system, [0,0])
        i_path[t] = max(0, params['phi_pi']*pi_path[t] + params['phi_y']*x_path[t])
    return pd.DataFrame({'Output Gap': x_path, 'Inflation': pi_path, 'Nominal Rate': i_path})

nk_params = nk_model.__dict__
T_sim = 60
# A shock of -0.05 means the natural rate falls by 5 percentage points.
r_n_shocks = np.zeros(T_sim); r_n_shocks[0] = -0.05

zlb_irfs = simulate_zlb(nk_params, r_n_shocks) * 100 # Convert to percent

zlb_irfs.plot(subplots=True, layout=(3,1), figsize=(10, 12), style='-o', markersize=4)
plt.suptitle('The Economy at the Zero Lower Bound', fontsize=16, y=1.0)
plt.tight_layout(); plt.show()
note("The large negative shock pushes the required nominal rate below zero. The central bank is constrained, and the output gap and inflation fall sharply. The economy only begins to recover as the natural rate returns to normal and the policy rate can lift off from zero.")

### 6. Chapter Summary
- **Core Idea:** New Keynesian models are DSGE models with nominal rigidities (usually sticky prices) that provide a microfoundation for monetary policy to have real effects.
- **Three Equations:** The canonical model consists of a Dynamic IS Curve (from household optimization), a New Keynesian Phillips Curve (from firm optimization with pricing frictions), and a monetary policy rule (e.g., a Taylor rule).
- **Optimal Policy:** The central bank can improve upon a simple rule by solving an optimal control problem to minimize a loss function over inflation and output gap volatility. This leads to an optimal targeting rule.
- **ZLB:** The Zero Lower Bound on nominal rates is a critical constraint on monetary policy. When it binds, the central bank may lose its ability to stabilize the economy, motivating unconventional policies like forward guidance.

### 7. Exercises

1.  **The Taylor Principle:** Rerun the baseline IRF simulation with a policy rule that violates the Taylor Principle (e.g., `phi_pi = 0.8`). What happens to the impulse responses? Does the model still have a unique, stable equilibrium? (You may need to solve the model analytically to see the instability).

2.  **Price Stickiness:** How does the economy's response to a monetary policy shock change if prices are stickier? Rerun the baseline IRF simulation with a higher Calvo parameter, `theta = 0.9`. How does this affect the slope of the Phillips curve, `kappa`, and the resulting dynamics of inflation and the output gap?

3.  **Optimal Policy Trade-off:** In the optimal policy problem, the parameter $\lambda_x$ in the loss function represents the central bank's relative weight on output stabilization vs. inflation stabilization. How would you expect the central bank's optimal response to a shock to change if $\lambda_x$ is very high? What if it's very low?

4.  **Forward Guidance at the ZLB:** In the ZLB simulation, suppose the central bank can credibly commit at time $t=0$ that it will keep the nominal rate at zero for two extra periods, even after the natural rate shock has dissipated. How would this commitment (a form of forward guidance) affect the paths of the output gap and inflation during the ZLB episode? Explain the mechanism.

5.  **Open Economy Extension:** Write down the system of equations for a small open economy NK model, including the UIP condition and the open economy IS curve. How does a domestic monetary tightening affect the nominal exchange rate in the short run and the long run?